In [126]:
import pandas as pd
from pandas.stats.api import ols
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import math
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
import math

from sklearn.cross_validation import cross_val_score, cross_val_predict
from sklearn import metrics

filepath = '/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/code/'

In [127]:
features = [
       'next_external_id', 
       'buzz_external_id', 
       'buzz_account_display_name', 
       'ext_table1_consumptions_by_type__link_clicks',
       'ext_table1_consumptions_by_type__other_clicks',
       'ext_table1_reactions_like_total',
       'ext_table1_stories_by_action_type__comment',
       'ext_table1_stories_by_action_type__like',
       'ext_table1_stories_by_action_type__share', 'ext_table1_engaged_fan',
       'ext_table1_fan_reach', 'ext_table1_impressions',
       'ext_table1_impressions_fan',
       'ext_twodays_consumptions_by_type__link_clicks',
       'ext_twodays_stories_by_action_type__share',
        ]

one_hour_features = [
                   'ext_table1_consumptions_by_type__link_clicks',
                   'ext_table1_consumptions_by_type__other_clicks',
                   'ext_table1_reactions_like_total',
                   'ext_table1_stories_by_action_type__comment',
                   'ext_table1_stories_by_action_type__like',
                   'ext_table1_stories_by_action_type__share', 
                   'ext_table1_engaged_fan',
                   'ext_table1_fan_reach',
                   'ext_table1_impressions',
                   'ext_table1_impressions_fan', 
                ]

two_day_features = [
    'ext_twodays_consumptions_by_type__link_clicks',
    'ext_twodays_stories_by_action_type__share',
]

# cross_promotion_features = all_pages

In [128]:
from sklearn import linear_model
clf = linear_model.LinearRegression()

print("LINEAR REGRESSION!")

train_df = pd.read_csv(filepath+'previous-to-next-page-info.csv')
all_pages = list(train_df.buzz_account_display_name.unique())
cross_promotion_features = all_pages

print("With log-scaled performance stats, and also cross-promotion features and page infos!")

page_infos = ['buzz_account_display_name', 'next_page']

# train_features = one_hour_features
train_features = one_hour_features + page_infos
# train_features = one_hour_features + page_infos + cross_promotion_features
# train_features = cross_promotion_features + one_hour_features
# train_features = two_day_features + one_hour_features + cross_promotion_features

for feats in (one_hour_features):
    train_df[feats] = np.log(train_df[feats] + 1)

# lm = linear_model.LinearRegression(fit_intercept=True, normalize=True)
cv = 10

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1)#.astype(int)
train_df['share_bucket'] = np.log(train_df['next_shares']+1)#.astype(int)

print("No. of data-points = ", len(train_df))

# y_clicks = train_df['next_clicks']
# y_share = train_df['next_shares']
y_clicks = train_df['clicks_bucket']
y_share = train_df['share_bucket']

###generate DFs for analysis - X and Y
X = train_df[train_features]
####convert page_infos into dummies
X = pd.get_dummies(data = X, columns=['buzz_account_display_name', 'next_page'])

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

for train_index, test_index in kf.split(X):
#     print("Fold = ", fold)
#     print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    ####Convert back to exponents
    y_pred = np.exp(y_pred) - 1
    y_test = np.exp(y_test) - 1
    ####
    
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/(np.exp(y) - 1).mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
#     print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    ####Convert back to exponents
    y_pred = np.exp(y_pred) - 1
    y_test = np.exp(y_test) - 1
    ####
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/(np.exp(y) - 1).mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

LINEAR REGRESSION!
With log-scaled performance stats, and also cross-promotion features and page infos!
No. of data-points =  18438
Shares performance: Accuracy =  0.474967002297  ; Error (/100) =  0.57346327498
Clicks performance: Accuracy =  0.70021002829  ; Error (/100) =  0.417612818109


In [132]:
######Test xgboost
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [114]:
# from sklearn import linear_model
# clf = linear_model.LinearRegression(fit_intercept=True,normalize=True)
# print("LINEAR REGRESSION!")

from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0, fit_intercept=True)
print("RIDGE REGRESSION! (L2-reg)")

# from sklearn.linear_model import Lasso
# clf = Lasso(alpha=1.0, fit_intercept = True)
# print("LASSO REGRESSION! (L1-reg)")

train_df = pd.read_csv(filepath+'previous-to-next-page-info.csv')
all_pages = list(train_df.buzz_account_display_name.unique())
cross_promotion_features = all_pages

print("With log-scaled performance stats, and also cross-promotion features and page infos!")

page_infos = ['buzz_account_display_name', 'next_page']

# train_features = one_hour_features
train_features = one_hour_features + page_infos
# train_features = one_hour_features + page_infos + cross_promotion_features
# train_features = cross_promotion_features + one_hour_features
# train_features = two_day_features + one_hour_features + cross_promotion_features

for feats in (one_hour_features):
    train_df[feats] = np.log(train_df[feats] + 1)

# lm = linear_model.LinearRegression(fit_intercept=True, normalize=True)
cv = 10

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1)#.astype(int)
train_df['share_bucket'] = np.log(train_df['next_shares']+1)#.astype(int)

print("No. of data-points = ", len(train_df))

# y_clicks = train_df['next_clicks']
# y_share = train_df['next_shares']
y_clicks = train_df['clicks_bucket']
y_share = train_df['share_bucket']

###generate DFs for analysis - X and Y
X = train_df[train_features]
####convert page_infos into dummies
X = pd.get_dummies(data = X, columns=['buzz_account_display_name', 'next_page'])

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

for train_index, test_index in kf.split(X):
#     print("Fold = ", fold)
#     print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    ####Convert back to exponents
    y_pred = np.exp(y_pred) - 1
    y_test = np.exp(y_test) - 1
    ####
    
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/(np.exp(y) - 1).mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
#     print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    ####Convert back to exponents
    y_pred = np.exp(y_pred) - 1
    y_test = np.exp(y_test) - 1
    ####
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/(np.exp(y) - 1).mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

RIDGE REGRESSION! (L2-reg)
With log-scaled performance stats, and also cross-promotion features and page infos!
No. of data-points =  18438
Shares performance: Accuracy =  0.474914539912  ; Error (/100) =  0.576142448304
Clicks performance: Accuracy =  0.710057086312  ; Error (/100) =  0.418992967428


In [110]:
###################(testing above!)
print(2)

2


In [111]:
from sklearn import linear_model
clf = linear_model.LinearRegression()
print("LINEAR REGRESSION!")

# from sklearn.tree import DecisionTreeRegressor
# clf = DecisionTreeRegressor(max_depth = 10)
# print("TREE REGRESSION!")

train_df = pd.read_csv(filepath+'previous-to-next-page-info.csv')
all_pages = list(train_df.buzz_account_display_name.unique())

print("Raw counts of performance stats with page names included!")
# print("WITH cross-promotion features")

page_infos = ['buzz_account_display_name', 'next_page']

# train_features = one_hour_features
train_features = one_hour_features + page_infos + cross_promotion_features
# train_features = one_hour_features + two_day_features + page_infos
# train_features = cross_promotion_features + one_hour_features

# train_features = two_day_features + one_hour_features + cross_promotion_features

# for feats in (one_hour_features):
#     train_df[feats] = np.log(train_df[feats] + 1)

# lm = linear_model.LinearRegression(fit_intercept=True, normalize=True)
cv = 10

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1).astype(int)
train_df['share_bucket'] = np.log(train_df['next_shares']+1).astype(int)

print("No. of data-points = ", len(train_df))

y_clicks = train_df['next_clicks']
y_share = train_df['next_shares']
# y_clicks = train_df['clicks_bucket']
# y_share = train_df['share_bucket']

###generate DFs for analysis - X and Y
X = train_df[train_features]
####convert page_infos into dummies
X = pd.get_dummies(data = X, columns=['buzz_account_display_name', 'next_page'])

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

for train_index, test_index in kf.split(X):
#     print("Fold = ", fold)
#     print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
        
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
#     print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/y.mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

LINEAR REGRESSION!
Raw counts of performance stats with page names included!
No. of data-points =  18438
Shares performance: Accuracy =  0.268176108801  ; Error (/100) =  1.08175611565
Clicks performance: Accuracy =  0.534925994061  ; Error (/100) =  0.741220995411


In [13]:
print("ONLY 1 HOUR PERFORMANCE IN PREVIOUS PAGE")
print('SHARES:')
res = ols(y = y_share, x = X)
print(res)
print('CLICKS:')
res = ols(y =y_clicks, x = X)
print(res)

ONLY 1 HOUR PERFORMANCE IN PREVIOUS PAGE
SHARES:


/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/pandas/stats/ols.py:336: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(np.diag(self._var_beta_raw))
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/eshwarchandrasekharan/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:875: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/eshwarchandrasekharan/anaconda3/


-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <ext_table1_consumptions_by_type__link_clicks>
             + <ext_table1_consumptions_by_type__other_clicks> + <ext_table1_reactions_like_total>
             + <ext_table1_stories_by_action_type__comment>
             + <ext_table1_stories_by_action_type__like> + <ext_table1_stories_by_action_type__share>
             + <ext_table1_engaged_fan> + <ext_table1_fan_reach>
             + <ext_table1_impressions> + <ext_table1_impressions_fan> + <SOML>
             + <Obsessed by BuzzFeed> + <Tasty> + <BuzzFeed> + <BuzzFeed Celeb> + <BuzzFeed News>
             + <BuzzFeed Quiz> + <BuzzFeed Style> + <BuzzFeed Australia>
             + <BuzzFeed Rewind> + <BuzzFeed Japan News> + <BuzzFeed Weddings>
             + <BuzzFeed Japan> + <BuzzFeed Video> + <BuzzFeed México> + <BuzzFeed Community>
             + <BuzzFeed Health> + <BuzzFeed Books> + <BuzzFeed World>
             + <BuzzFeed Food> + <Bu

In [41]:
######ALT - label encoding for getting the current and next pages - 
one_hour_features = [
                   'ext_table1_consumptions_by_type__link_clicks',
                   'ext_table1_consumptions_by_type__other_clicks',
                   'ext_table1_reactions_like_total',
                   'ext_table1_stories_by_action_type__comment',
                   'ext_table1_stories_by_action_type__like',
                   'ext_table1_stories_by_action_type__share', 
                   'ext_table1_engaged_fan',
                   'ext_table1_fan_reach',
                   'ext_table1_impressions',
                   'ext_table1_impressions_fan', 
                ]

from sklearn import linear_model
clf = linear_model.LinearRegression()

print("LINEAR REGRESSION!")

train_df = pd.read_csv(filepath+'previous-to-next-page-info.csv')
all_pages = list(train_df.buzz_account_display_name.unique())
cross_promotion_features = all_pages

###encode page info - labels
from sklearn import preprocessing
curr_le = preprocessing.LabelEncoder()
curr_le = curr_le.fit(train_df['buzz_account_display_name'])
train_df['current_page_label'] = curr_le.transform(train_df['buzz_account_display_name'])
next_le = preprocessing.LabelEncoder()
next_le = next_le.fit(train_df['next_page'])
train_df['next_page_label'] = next_le.transform(train_df['next_page'])

print("With log-scaled performance stats, and also cross-promotion features and page infos!")

# page_infos = ['buzz_account_display_name', 'next_page']
page_infos = ['current_page_label', 'next_page_label']

train_features = one_hour_features
include_page_info = 1

if include_page_info == 1:
    train_features = one_hour_features + page_infos
# train_features = one_hour_features + page_infos + cross_promotion_features
# train_features = cross_promotion_features + one_hour_features
# train_features = two_day_features + one_hour_features + cross_promotion_features

logscale = 1

if logscale == 1:
    for feats in (one_hour_features):
        train_df[feats] = np.log(train_df[feats] + 1)

# lm = linear_model.LinearRegression(fit_intercept=True, normalize=True)
cv = 10

train_df['clicks_bucket'] = np.log(train_df['next_clicks'] + 1)#.astype(int)
train_df['share_bucket'] = np.log(train_df['next_shares']+1)#.astype(int)

print("No. of data-points = ", len(train_df))

# y_clicks = train_df['next_clicks']
# y_share = train_df['next_shares']
y_clicks = train_df['clicks_bucket']
y_share = train_df['share_bucket']

###generate DFs for analysis - X and Y
X = train_df[train_features]
####convert page_infos into dummies
# X = pd.get_dummies(data = X, columns=['buzz_account_display_name', 'next_page'])

from sklearn.model_selection import KFold

fold = 0
cv_folds = 10
kf = KFold(n_splits = cv_folds, shuffle = True)

clicks_accuracy = []
clicks_error = []
shares_accuracy = []
shares_error = []

for train_index, test_index in kf.split(X):
#     print("Fold = ", fold)
#     print("Shares: Fold = ", fold)
    y = y_share
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    ####Convert back to exponents
    y_pred = np.exp(y_pred) - 1
    y_test = np.exp(y_test) - 1
    ####
    
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    shares_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/(np.exp(y) - 1).mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    shares_error.append(error_percent)
    
#     print("Clicks: Fold = ", fold)
    y = y_clicks
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    
    ####Convert back to exponents
    y_pred = np.exp(y_pred) - 1
    y_test = np.exp(y_test) - 1
    ####
    accuracy = metrics.r2_score(y_test, y_pred)
#     print("Cross-Predicted Accuracy (R2):", accuracy)
    clicks_accuracy.append(accuracy)
    from sklearn.metrics import mean_absolute_error
    # print("Mean Absolute Error: ", mean_absolute_error(y, predictions))
    error_percent = mean_absolute_error(y_test, y_pred)/(np.exp(y) - 1).mean()
#     print("Mean values (share): ", y_test.mean(), " | percent error: ",  error_percent)
    clicks_error.append(error_percent)
    fold += 1
print("Shares performance: Accuracy = ", np.mean(shares_accuracy), " ; Error (/100) = ", np.mean(shares_error))
print("Clicks performance: Accuracy = ", np.mean(clicks_accuracy), " ; Error (/100) = ", np.mean(clicks_error))

LINEAR REGRESSION!
With log-scaled performance stats, and also cross-promotion features and page infos!
No. of data-points =  18438
Shares performance: Accuracy =  0.0424335022281  ; Error (/100) =  0.882574670839
Clicks performance: Accuracy =  -0.00291987853618  ; Error (/100) =  0.896657782014


In [19]:
train_features

['ext_table1_consumptions_by_type__link_clicks',
 'ext_table1_consumptions_by_type__other_clicks',
 'ext_table1_reactions_like_total',
 'ext_table1_stories_by_action_type__comment',
 'ext_table1_stories_by_action_type__like',
 'ext_table1_stories_by_action_type__share',
 'ext_table1_engaged_fan',
 'ext_table1_fan_reach',
 'ext_table1_impressions',
 'ext_table1_impressions_fan',
 'current_page_label',
 'next_page_label']

In [86]:
train_df = pd.read_csv(filepath+'previous-to-next-page-info.csv')
all_pages = list(train_df.buzz_account_display_name.unique())
cross_promotion_features = all_pages
train_df = train_df[one_hour_features + ['buzz_account_display_name', 'next_page']]
all_pages = train_df['buzz_account_display_name'].unique()
train_df['next_page'] = train_df['next_page'].astype('category', categories = all_pages)
train_df['buzz_account_display_name'] = train_df['buzz_account_display_name'].astype('category', categories = all_pages)
###encode page info - labels
# from sklearn import preprocessing
# curr_le = preprocessing.LabelEncoder()
# curr_le = curr_le.fit(train_df['buzz_account_display_name'])
# train_df['current_page_label'] = curr_le.transform(train_df['buzz_account_display_name'])
# next_le = preprocessing.LabelEncoder()
# next_le = next_le.fit(train_df['next_page'])
# train_df['next_page_label'] = next_le.transform(train_df['next_page'])

############
# X = pd.get_dummies(data = X, columns=['buzz_account_display_name', 'next_page'])
train_df

,ext_table1_consumptions_by_type__link_clicks,ext_table1_consumptions_by_type__other_clicks,ext_table1_reactions_like_total,ext_table1_stories_by_action_type__comment,ext_table1_stories_by_action_type__like,ext_table1_stories_by_action_type__share,ext_table1_engaged_fan,ext_table1_fan_reach,ext_table1_impressions,ext_table1_impressions_fan,buzz_account_display_name,next_page
0,4757.0,628.0,57,4.0,90.0,13.0,4764,56919,74768,73608,SOML,BuzzFeed UK
1,637.0,85.0,62,6.0,68.0,12.0,678,14639,20178,19485,Obsessed by BuzzFeed,BuzzFeed
2,80435.0,9881.0,1350,614.0,1574.0,137.0,83121,2496210,3210276,3162757,Tasty,BuzzFeed Canada
3,596.0,115.0,31,12.0,43.0,11.0,726,38871,47788,47082,SOML,BuzzFeed Video
4,33360.0,17868.0,2121,159.0,5346.0,551.0,39527,457943,676049,592051,BuzzFeed,Cocoa Butter
5,111.0,24.0,0,4.0,0.0,0.0,125,5834,7219,7119,BuzzFeed Celeb,Obsessed by BuzzFeed
6,16669.0,650.0,102,16.0,110.0,14.0,15548,246819,314136,311217,BuzzFeed,Buy Me That
7,4102.0,1971.0,394,49.0,655.0,41.0,5462,80576,118176,104353,BuzzFeed News,BuzzFeed Australia
8,2149.0,130.0,44,28.0,47.0,5.0,2017,22104,30526,29782,BuzzFeed Quiz,BuzzFeed Quiz
9,3295.0,302.0,352,13.0,422.0,63.0,3527,53100,69751,63998,BuzzFeed Style,BuzzFeed Rewind
